# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",  
    frame_width = 800,
    frame_height = 600,
    size = "City_Population",
    scale = 0.05,
    color = "City",  
    alpha = 0.7,
    hover_cols = ["City", "Country", "City_Population"] 
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria (updated to your conditions)
filtered_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
filtered_cities_df.dropna(inplace=True)

# Display sample data (without tabulate)
print("Filtered Cities:")
print("-" * 20)
if not filtered_cities_df.empty:  
    for index, row in filtered_cities_df.head().iterrows():
        print(f"City: {row['City']}, Country: {row['Country']}, Max Temp: {row['Max Temp']}, Wind Speed: {row['Wind Speed']}, Cloudiness: {row['Cloudiness']}")
else:
    print("No cities found that match the criteria.")


Filtered Cities:
--------------------
City: kapaa, Country: US, Max Temp: 22.99, Wind Speed: 3.6, Cloudiness: 0
City: hilo, Country: US, Max Temp: 26.27, Wind Speed: 2.57, Cloudiness: 0
City: banda, Country: IN, Max Temp: 24.62, Wind Speed: 2.68, Cloudiness: 0
City: makakilo city, Country: US, Max Temp: 21.66, Wind Speed: 2.57, Cloudiness: 0
City: kahului, Country: US, Max Temp: 23.8, Wind Speed: 3.09, Cloudiness: 0


C:\Users\joesa\AppData\Local\Temp\ipykernel_7796\1708783719.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_cities_df.dropna(inplace=True)


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Create a copy of the filtered DataFrame for hotels
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()  

# Add an empty column for Hotel Name
hotel_df["Hotel Name"] = ""

# Display the first 5 rows of hotel_df
print("Potential Hotel Locations:")
print("-" * 20)
for index, row in hotel_df.head().iterrows():
    print(f"City: {row['City']}, Country: {row['Country']}, Lat: {row['Lat']}, Lng: {row['Lng']}, Humidity: {row['Humidity']}, Hotel Name: {row['Hotel Name']}")


Potential Hotel Locations:
--------------------
City: kapaa, Country: US, Lat: 22.0752, Lng: -159.319, Humidity: 84, Hotel Name: 
City: hilo, Country: US, Lat: 19.7297, Lng: -155.09, Humidity: 83, Hotel Name: 
City: banda, Country: IN, Lat: 25.4833, Lng: 80.3333, Humidity: 52, Hotel Name: 
City: makakilo city, Country: US, Lat: 21.3469, Lng: -158.0858, Humidity: 81, Hotel Name: 
City: kahului, Country: US, Lat: 20.8947, Lng: -156.47, Humidity: 60, Hotel Name: 


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel within 10,000 meters
radius = 10000  
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,                         # Get only the first result
    "apiKey": geoapify_key              # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search...")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}" 
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
print("\nHotel Results:")
print("-" * 20)
for index, row in hotel_df.head().iterrows():
    print(f"City: {row['City']}, Country: {row['Country']}, Lat: {row['Lat']}, Lng: {row['Lng']}, Humidity: {row['Humidity']}, Hotel Name: {row['Hotel Name']}")


Starting hotel search...
kapaa - nearest hotel: Pono Kai Resort
hilo - nearest hotel: Dolphin Bay Hotel
banda - nearest hotel: #acnindiafy21
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
kahului - nearest hotel: Maui Seaside Hotel
gat - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
tikaitnagar - nearest hotel: No hotel found
san quintin - nearest hotel: Jardines Hotel
santa rosalia - nearest hotel: Sol y Mar
narwar - nearest hotel: No hotel found
port hedland - nearest hotel: The Esplanade Hotel
roebourne - nearest hotel: No hotel found
saint-francois - nearest hotel: Chez Lily
capoterra - nearest hotel: Rosa Hotel
stolac - nearest hotel: Bregava
guerrero negro - nearest hotel: Plaza sal paraiso

Hotel Results:
--------------------
City: kapaa, Country: US, Lat: 22.0752, Lng: -159.319, Humidity: 84, Hotel Name: Pono Kai Resort
City: hilo, Country: US, Lat: 19.7297, Lng: -155.09, Humidity: 83, Hotel Name: Dolphin Bay Hotel
City

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
%%capture --no-display
import hvplot.pandas

# Configure the map plot for cities
city_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=800,
    frame_height=600,
    size=15, 
    color="City",
    hover_cols=["City", "Country", "Hotel Name"] 
)

# Filter to hotels found
hotel_df_found = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]

# Configure the map plot for hotels
hotel_plot = hotel_df_found.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles='OSM',
    frame_width=800,
    frame_height=600,
    size=15,
    color="green",
    marker="star",
    hover_cols=["City", "Country", "Hotel Name"]  
)
# Combine both plots
combined_plot = city_plot * hotel_plot
# Display the map
combined_plot


:Overlay
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Points.I  :Points   [Lng,Lat]   (City,Country,Hotel Name)
   .WMTS.II   :WMTS   [Longitude,Latitude]
   .Points.II :Points   [Lng,Lat]   (City,Country,Hotel Name)